In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import time

import pickle
import os,errno


first set of experiments  = exp-prop-K-
 -> capacity load for the gut not individual bacteria. 

 second set of experiments = exp-prop-K-3-
 --> capacity load for gut with 3 bacteria only

 third set of experiments = by varying gc, dc 
 --> various combinations of gc and dc 
 

# Before copy the plot data from latest file, ensure removing the object references - as they will be passed in the data pickle. 

In [2]:
def generatePlotSel(choice = 'random',**kwargs):
    sel = np.arange(5)
    if choice == 'random':
        n = kwargs.get('n',5)
        randSel = np.random.randint(0,iterations,n)
        randSel.sort()
        sel =  randSel
    elif choice == 'all':
        sel = np.arange(iterations)
    elif choice == 'custom':
        start = kwargs.get('start',0)
        stop = kwargs.get('stop',5)
        step = kwargs.get('step',1)
        sel = np.arange(start,stop,step)
    
    else:
        print("first five selection only.")
    return sel

def createDir(dir):
  try:
      os.mkdir(dir)
      print("created directory",dir)
  except OSError as e:
      if e.errno == errno.EEXIST:
          print('Directory ',dir,'already exists.')
      else:
          raise  


In [3]:
def plotPopulation():
    # one scatter plot for each bacterium population. 
    # relevant to trace out the population evolution across timesteps.

    X = np.arange(sel.size)
    fig,ax = plt.subplots(nrows,ncols,sharex = True, sharey = True,figsize = (15,6))
    cm = plt.get_cmap("tab10")
    ax = ax.ravel()

    for i in range(nb):
      ax[i].scatter(X,gutPopHistory[sel,i],color = cm(i))
      ax[i].set_ylabel("gut population")
      ax[i].set_xlabel("time")
      ax[i].text(0.5, 0.5, contribution[i], horizontalalignment='center', verticalalignment='center',     transform=ax[i].transAxes)
      ax[i].set_title("""contribution = {0} [init pop = {1}]""".format(contribution[i],initial_pop[i]))

    fig.text(0.4,0.9,"""Individual gut bacteria population evolution
  gc = {0:.2f},dc = {1:.2f}, exploration decay = {2}; carrying capacity = {3}
  """.format(gc,dc,epsilon_mod,carrying_capacity))

    if saveFig:
      plt.savefig(folder+'scatter-per-bact-pop'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format= fileFormat)
    plt.close()


# %%
def plotState():
  #@title
  # one scatter plot for state history. 
  # relevant to trace out the population evolution across timesteps.


  X = np.arange(sel.size)
  fig,ax = plt.subplots(nrows,ncols,sharex = True, sharey = True,figsize = (15,6))
  cm = plt.get_cmap("tab10")
  ax = ax.ravel()

  for i in range(nb):
    ax[i].scatter(X,stateHistory[sel,i],color = cm(i))
    ax[i].set_ylabel("cravings")
    ax[i].set_xlabel("time")
    ax[i].text(0.5, 0.5, contribution[i], horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes)
    ax[i].set_title("""contribution = {0}""".format(contribution[i]))

  fig.text(0.4,0.9,"""Individual state history
  gc = {0:.2f},dc = {1:.2f}; exploration_decay = {2}
  """.format(gc,dc,epsilon_mod))
  if saveFig:
    plt.savefig(folder+'scatter-state'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format= fileFormat)    
  plt.close()


# %%
#@title
def plotReward():

  # plotting reward history
  plt.scatter(range(sel.size),rewardHistory[sel])
  plt.xlabel("time")
  plt.ylabel("normalized reward")
  if saveFig:
    plt.savefig(folder+'reward'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format= fileFormat)
  plt.close()        


# %%
#@title
def plotBehavior():
  # Behavior history for each bacterium - whether the bacteria received the nutrient or not. 
  # yellow represents higher values (or presence of nutrient to the bacteria)
  # width of heat map = 
  # nrow = 2; ncol = gut.nBacteria//nrow

  a = behaviorHistory
  height = int(0.9*len(stateSet)//len(actionSet)) #height of the heat map
  # height = int(len(behavior.stateSet)//2)
  fig,ax = plt.subplots(nrows,ncols,sharey = True,sharex = True,figsize = (50,50))
  ax = ax.ravel()

  for i in range(nb):
    # ax[i].imshow(np.tile(a[plotN:,i],(height,1)))
    ax[i].imshow(np.tile(a[sel,i],(height,1)))
    ax[i].set_xticklabels([])
    ax[i].set_yticklabels([])
    # ax[i].set_title("nutrient "+str(i+1))
    ax[i].set_title("""nutrient {0} contri = {1}""".format(i+1, contribution[i]))

  fig.text(0.3,0.3,"time steps (whether with time, nutrients were being ingested or not?)")
  if saveFig:
    plt.savefig(folder+'nutrient-behavior'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format= fileFormat)
  plt.close()


# %%
#@title
#heat map
def plotQTable():
    nstates = len(stateSet)
    nactions = len(actionSet)
    expansion = 0.5*nstates//nactions


    actions = actionSet
    states = [stateSet[int(i)] for i in np.linspace(0,nstates,stateSlicer+1)[:-1]]

    fig,ax = plt.subplots(1,1,figsize = (20,20))
    ax.imshow(np.repeat(qTable,expansion,1))

    ax.set_xticks(np.linspace(0,nactions*expansion-1,nactions+1))
    ax.set_xticklabels(actions,rotation = 45,fontsize = 6)

    ax.set_yticks(np.linspace(0,nstates,stateSlicer+1))
    ax.set_yticklabels(states,rotation = 90,fontsize = 6)

    ax.set_xlabel("actions")
    ax.set_ylabel("states")

    if saveFig:
      plt.savefig(folder+'qtable'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format = fileFormat)
    plt.close()        


# %%
#@title
def plotActionHistory():
  plt.clf()
  # plot action history to check the stagnation towards the end

  X = np.arange(sel.size)

  fig,ax = plt.subplots(1,1,sharex = True, sharey = True,figsize = (15,6))
  cm = plt.get_cmap("tab10")

  act_ind = []
  # actions = behavior.actionSet
  actions = actionSet
  for a in actionHistory[sel]:
    act_ind.append(np.where((a == actions).all(1))[0][0])


  plt.scatter(X,act_ind,c = np.tile(range(20),len(X)//20),cmap = plt.get_cmap())
  plt.ylabel("Action")
  plt.xlabel("Time")
  plt.colorbar()
  plt.yticks(range(len(actions)),actions) 
  fig.text(0.5,0.9,"Action History")

  if saveFig:
    plt.savefig(folder+'action-history'+str(counter+1)+'.'+fileFormat,bbox_inches = 'tight',format= fileFormat)
  plt.close()


# CHANGE THE STUFF IN BELOW CELL TO GET THINGS GOING, AFTER COPYING DATA FROM SERVER 

In [6]:
num_simulations = 3
num_experiments = 4
num_growth_exp = 4

ff = 'growth experiments-2/'
base_dir = '/home/anjali/Documents/Code/Jupyter/GutProject/GutToyModel/simulations/simulations-28May/'

# exp_typ ='exp-prop-K-'
# exp_typ = 'exp-prop-K-3-'


In [8]:

saveFig = 'True'
for g_exp in range(4,6):
    for exp_num in range(num_experiments):
        folder_data = base_dir+ff+'grow-'+str(g_exp+1)+'-3b-contri-'+str(exp_num+1)+'/data/'
        # folder_data = base_dir+exp_typ+str(exp_num+1)+'/data/'
        # folder_data = base_dir+str(exp_num+1)+'/data/'

        folder = folder_data+'../'
        


        for cc in range(num_simulations):


            with open(folder_data+'plotData-'+str(cc+1)+'.pickle','rb') as handle: 
                plotData = pickle.load(handle)

            actionHistory = np.load(folder_data+'actionHistory-'+str(cc+1)+'.npy')
            stateHistory = np.load(folder_data+'stateHistory-'+str(cc+1)+'.npy')
            behaviorHistory = np.load(folder_data+'behaviorHistory-'+str(cc+1)+'.npy')
            rewardHistory = np.load(folder_data+'rewardHistory-'+str(cc+1)+'.npy')
            qtable = np.load(folder_data+'qtable-'+str(cc+1)+'.npy')
            gutPopHistory = np.load(folder_data+'gutPopHistory-'+str(cc+1)+'.npy')


            for key,val in plotData.items():
                exec(key + '=val')

            choice = 'random'
            plotDataPoints = 500
            start,stop,step = 0,5,1
            sel = generatePlotSel(choice,n = plotDataPoints,iterations = iterations)  #check code 

            plotActionHistory()
            plotPopulation()
            plotState()
            plotReward()
            plotBehavior()
            plotQTable()
            
